# Fashion MNIST classification (cont.)



This notebook is built upon the [TensorFlow image classification tutorial](https://www.tensorflow.org/tutorials/keras/classification), which trains a neural network model to classify Fashion MNIST dataset. It's okay if you don't understand all the details of the implementation; this is a fast-paced overview of a complete TensorFlow program with the details explained as you go. We use [tf.keras](https://www.tensorflow.org/guide/keras), a high-level API to build and train models in TensorFlow.

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

# set random seeds
np.random.seed(42)
tf.random.set_seed(42)
tf.compat.v1.set_random_seed(42)
tf.experimental.numpy.random.seed(42)
print(tf.__version__)

## Preprocess the data as in the starter code

### Import the Fashion MNIST dataset

You can access the Fashion MNIST directly from TensorFlow. Import and load the Fashion MNIST data directly from TensorFlow.

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

Loading the dataset returns four NumPy arrays:

* The `train_images` and `train_labels` arrays are the *training set*—the data the model uses to learn.
* The model is tested against the *test set*, the `test_images`, and `test_labels` arrays.

We will split a validation set from the training set later.

### Scale and flatten the images

The images are 28x28 NumPy arrays, with pixel values ranging from 0 to 255. Scale these values to a range of 0 to 1 before feeding them to the neural network model. To do so, divide the values by 255. Moreover, flatten the 28x28 NumPy arrays into 784 dimension feature vectors. 

It's important that the *training set* and the *testing set* be preprocessed in the same way.

In [ ]:
train_images = (train_images / 255.0).reshape(train_images.shape[0], -1)
test_images = (test_images / 255.0).reshape(test_images.shape[0], -1)
train_images.shape

The *labels* are an array of integers, ranging from 0 to 9. Each image is mapped to a single label. Since the *class names* are not included with the dataset, store them here to use later when plotting the images:

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

### Split train/validation/test sets

Same as the starter code, take the first 10k data in the *training set* as the training data, and the last 2k data in the *training set* as the validation data. The test data is all the 10k data in the *testing set*.

In [ ]:
# sample 5k training, 1k validation, making sure the label is balanced
label_idxs = [np.where(train_labels == i)[0] for i in range(10)]
Xtrain, Ytrain, Xval, Yval = \
    np.concatenate([train_images[label_idxs[i][:1000]] for i in range(10)]), \
    np.concatenate([train_labels[label_idxs[i][:1000]] for i in range(10)]), \
    np.concatenate([train_images[label_idxs[i][-200:]] for i in range(10)]), \
    np.concatenate([train_labels[label_idxs[i][-200:]] for i in range(10)])

# shuffle data before training
idx_order = np.random.permutation(Xtrain.shape[0])
Xtrain = Xtrain[idx_order]
Ytrain = Ytrain[idx_order]

Xtest, Ytest = test_images, test_labels

To verify that the data is in the correct format and that you're ready to build and train the network, let's display the first 25 images from the training data and display the class name below each image.

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(Xtrain[i].reshape(28, 28), cmap=plt.cm.binary)
    plt.xlabel(class_names[Ytrain[i]])
plt.show()

## Build the model

Building the neural network requires configuring the layers of the model, then compiling the model.

### Set up two linear layers

The basic building block of a neural network is the [*layer*](https://www.tensorflow.org/api_docs/python/tf/keras/layers). Layers extract representations from the data fed into them. Hopefully, these representations are meaningful for the problem at hand.

Most of deep learning consists of chaining together simple layers. Most layers, such as `tf.keras.layers.Dense`, have parameters that are learned during training.

In [ ]:
base_model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

The network consists of a sequence of two `tf.keras.layers.Dense` layers. These are densely connected, or fully connected, neural layers. The first `Dense` layer has 1000 nodes (or neurons). The second layer returns a logits array with length of 10. Each node contains a score that indicates the current image belongs to one of the 10 classes.

### Compile the model

Before the model is ready for training, it needs a few more settings. These are added during the model's [*compile*](https://www.tensorflow.org/api_docs/python/tf/keras/Model#compile) step:

* [*Loss function*](https://www.tensorflow.org/api_docs/python/tf/keras/losses) —This measures how accurate the model is during training. You want to minimize this function to "steer" the model in the right direction.
* [*Optimizer*](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) —This is how the model is updated based on the data it sees and its loss function.
* [*Metrics*](https://www.tensorflow.org/api_docs/python/tf/keras/metrics) —Used to monitor the training and testing steps. The following example uses *accuracy*, the fraction of the images that are correctly classified.

In [ ]:
base_model.compile(optimizer='sgd',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

### Setup early-stopping

Define the early-stopping module as a callback module to stop the training process.

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor="val_accuracy",
        # "no longer improving" being defined as "for at least 3 epochs"
        patience=3,
        verbose=1,
    )
]

## Train the model

Training the neural network model requires the following steps:

1. Feed the training data to the model. In this example, the training data is in the `Xtrain` and `Ytrain` arrays.
2. The model learns to associate images and labels.
3. You ask the model to make predictions about a test set—in this example, the `Xtest` array.
4. Verify that the predictions match the labels from the `Ytest` array.


### Feed the model

To start training,  call the [`model.fit`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit) method—so called because it "fits" the model to the training data. It automatically processes the forward pass and the backward pass.

---



In [ ]:
base_model.fit(
    Xtrain, Ytrain, 
    batch_size=5, epochs=50, 
    callbacks=callbacks,
    validation_data=(Xval, Yval))

### Evaluate accuracy

Next, compare how the model performs on the test dataset:

In [ ]:
test_loss, test_acc = base_model.evaluate(Xtest,  Ytest, verbose=2)
print('\nTest accuracy:', test_acc)

**The accuracy that we obtain here is similar (+/- 1.5) to the accuracy from your own Python code for a batch size of 5. You can check your results to verify this.**

## Problem 3.3: The effect of early-stopping

We provided built-in early-stopping in the starter code, but did not explore the effect that it has on training. In this question, we will explore the effect of early-stopping. To do this, we evaluate the training, validation and test accuracy for each training epoch. We then plot the training, validation and test accuracy throughout the training process to see how early-stopping works.

To make sure the training process is the same as the previous `base_model`, we set the same random seed for training. Connecting all the training-evaluation modules together, we train 30 epochs on a batch size of 5 without early-stopping:

In [ ]:
# set the same random seed to make sure the model initialized at the same weights.
tf.random.set_seed(42)
tf.compat.v1.set_random_seed(42)
tf.experimental.numpy.random.seed(42)
# build the model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])
test_acc = []
# setup a callback for evaluate the test set
class TestCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        res_test = self.model.evaluate(Xtest, Ytest, verbose = 0)
        test_acc.append(res_test[1])
my_test_callback = TestCallback()
# compile the model
model.compile(optimizer='sgd',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
# train the model without callbacks
train_test_history = model.fit(
    Xtrain, Ytrain, 
    batch_size=5, epochs=30,
    callbacks=[my_test_callback],
    validation_data=(Xval, Yval))

### Visualize the accuracy without early-stopping

Run the provided code to plot the training accuracy *vs.* the number of epochs. On the same graph, we plot the test accuracy *vs.* the number of epochs. To do this, we retrieve loss and accuracy information in `train_test_history.history`. We also use `plt.scatter` to mark the point on the test accuracy curve where we early-stopped previously.

In [ ]:
x = np.arange(len(train_test_history.history['accuracy']))
train_acc = train_test_history.history['accuracy']
val_acc = train_test_history.history['val_accuracy']

plt.title("Train-val-test accuracy through the epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.plot(x, train_acc, label="train acc")
plt.plot(x, val_acc, label="val acc")
plt.plot(x, test_acc, label="test acc")
plt.scatter([x[12]], [val_acc[12]], color="red")
plt.legend()
plt.show()

Based on the curve, answer the following questions:

*   What is the trend of training and test accuracy after the early-stopped point?
*   Based on the plot, what do you think could go wrong if the patience parameter for early-stopping is too small?



## Problem 3.4: SGD with momentum



We mentioned in the lecture that adding a “momentum” term, which encourages model to continue along previous gradient direction, helps the network to converge. Concretely, with an initial velocity $v=0$, we update the gradient by

$v ← \alpha v + \nabla F(w)$

$w ← w - \eta v$

where $\eta$ is the learning rate and $\alpha$ is the factor describing how much weight we put on the previous gradients. $\alpha=0$ is equivalent to gradient update without momentum. 

We provide the `sgd_with_momentum` function with argument $\alpha$ below. In this question, we will explore the  momentum factor $\alpha$ effects training loss and test accuracy.

In [ ]:
def sgd_with_momentum(alpha):
  # set the same random seed to make sure the model initialized at the same weights.
  tf.random.set_seed(42)
  tf.compat.v1.set_random_seed(42)
  tf.experimental.numpy.random.seed(42)
  # build the model
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10)
  ])
  # compile the model
  model.compile(optimizer=tf.keras.optimizers.SGD(momentum=alpha),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  # train the model without early-stopping
  history = model.fit(
      Xtrain, Ytrain, 
      batch_size=5, epochs=30)
  # evaluate the model
  test_loss, test_acc = model.evaluate(Xtest,  Ytest, verbose=2)
  # return training loss and test accuracy
  return history.history['loss']

### Visualize training loss and test accuracy with different momentum factor $\alpha$

We will explore how different momentum factor $\alpha$ effects training loss and test accuracy. Call the `sgd_with_momentum` function with $\alpha$ = 0.1, 0.3, 0.5, 0.7, 0.9. Store the training loss and test accuracy from the return of the function.

In [ ]:
train_losses = []

for alpha in np.arange(0.1, 1, 0.2):
  train_loss = sgd_with_momentum(alpha)
  train_losses.append(train_loss)

We visualize the training loss. Plot 5 curve of different $\alpha$ on the same graph. Each curve has epoch number as the $x$-axis and the training loss as the $y$-axis.  Based on this, what is a suitable value of $\alpha$? Therefore, how should training ideally rely on previous gradients for better convergence?

In [ ]:
x = np.arange(len(train_losses[0]))

plt.title("Training loss through the epochs for different SGD momentum")
plt.xlabel("Epoch")
plt.ylabel("Loss")
for i, alpha in enumerate(np.arange(0.1, 1, 0.2)):
  plt.plot(x, train_losses[i], label="momentum=%f" % alpha)
plt.legend()
plt.show()

## [Bonus] Problem 3.5: Out-of-domain generalization

A major research direction at present is to ensure that our ML models not only do well on test data drawn from the same distribution as training data, but also do well when they get data from distributions different from the original distribution. This is known as Out-of-Distribution (OOD) generalization. In the Fashion MNIST dataset, the training and test set are drawn from the same distribution. Let's explore how our models do on test data coming from a slightly different distribution.


### Create OOD test set by translation

Take a look at the original test set of Fashion MNIST.

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(Xtest[i].reshape(28, 28), cmap=plt.cm.binary)
    plt.xlabel(class_names[Ytest[i]])
plt.show()

We move up the images of the test set by 4 pixels, and create a new translated test set from that. You will notice that to a human eye, the new test set is not any more difficult than the original test set.

In [ ]:
# lift the image by 4 pixels
translate_x = 4
translate_y = 0

Xtest_t4 = np.pad(Xtest.reshape(-1, 28, 28)[:, translate_x:, translate_y:], 
                  ((0, 0), (0, translate_x), (0, translate_y))).reshape(-1, 784)


# visualize the translation
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(Xtest_t4[i].reshape(28, 28), cmap=plt.cm.binary)
    plt.xlabel(class_names[Ytest[i]])
plt.show()

*But can the MLP model still identify it well?* Check the test accuracy on the base model.

In [ ]:
test_loss, test_acc = base_model.evaluate(Xtest_t4,  Ytest, verbose=2)
print('\nTest accuracy:', test_acc)

### Convolutional Neural Network

What if we try a different model architecture? For example, in class we saw that convolutional neural networks are good at dealing with image translation. We replace the first linear layer with a convolutional layer of 64 kernels with size $7\times7$, followed by max-pooling. This can be done with just one or two lines of code in TensorFlow. We train the 2-layer CNN on the original training data and test it on both the original and the translated test sets.

In [ ]:
# set the same random seed.
tf.random.set_seed(42)
tf.compat.v1.set_random_seed(42)
tf.experimental.numpy.random.seed(42)
# replace the linear layer with a conv layer of 64 7*7 kernels, followed by maxpooling.
cnn_model = tf.keras.Sequential([
    tf.keras.layers.Reshape((28, 28, 1), input_shape=(784,)),
    tf.keras.layers.Conv2D(64, (7, 7), activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)
])
# compile the model
cnn_model.compile(optimizer='sgd',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
# train the model with early-stopping
cnn_model.fit(
    Xtrain, Ytrain, 
    batch_size=5, epochs=50,
    callbacks=callbacks,
    validation_data=(Xval, Yval))
# evaluate on the original test set
test_loss, test_acc = cnn_model.evaluate(Xtest,  Ytest, verbose=2)
print('In-domain test accuracy:', test_acc)
# evaluate on the translated test set
test_loss, test_acc = cnn_model.evaluate(Xtest_t4,  Ytest, verbose=2)
print('Out-of-domain test accuracy:', test_acc)

Show the CNN model has fewer parameters (compute by hand, you can use 
`cnn_model.trainable_variables` to verify your calculation) than the 2-layer 
MLP. What is the in-domain test accuracy and the translated test accuracy?




### Make the CNN Deeper

Going one step further, we can make the CNN deeper by adding one more convolutional layer between the two existing layers. 

In [ ]:
# set the same random seed.
tf.random.set_seed(42)
tf.compat.v1.set_random_seed(42)
tf.experimental.numpy.random.seed(42)
# add one more conv layer of 128 2*2 kernels in between, followed by maxpooling.
cnn_model_deeper = tf.keras.Sequential([
    tf.keras.layers.Reshape((28, 28, 1), input_shape=(784,)),
    tf.keras.layers.Conv2D(64, (7, 7), activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, (2, 2), activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10)
])
# compile the model
cnn_model_deeper.compile(optimizer='sgd',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
# train the model with early-stopping
cnn_model_deeper.fit(
    Xtrain, Ytrain, 
    batch_size=5, epochs=50,
    callbacks=callbacks,
    validation_data=(Xval, Yval))
# evaluate on the original test set
test_loss, test_acc = cnn_model_deeper.evaluate(Xtest,  Ytest, verbose=2)
print('In-domain test accuracy:', test_acc)
# evaluate on the translated test set
test_loss, test_acc = cnn_model_deeper.evaluate(Xtest_t4,  Ytest, verbose=2)
print('Out-of-domain test accuracy:', test_acc)

Show the deeper 3-layer CNN model has fewer parameters (compute by hand, you can use `cnn_model.trainable_variables` to verify your calculation) than the 2-layer CNN model. What is the in-domain and the translated test accuracy of the deeper CNN model? Provide some intuition on why it is better than the 2-layer CNN on the translated set.

### Create OOD test set by rotation



Next, we create 3 more OOD test sets by rotation. We rotate the images in the original test set by 90, 180 and 270 degrees.

In [ ]:
# rotate 90 degrees counter-clockwise 
Xtest_r90 = np.rot90(Xtest.reshape(-1, 28, 28), k=1, axes=(1, 2)).reshape(-1, 784)

# rotate 180 degrees counter-clockwise 
Xtest_r180 = np.rot90(Xtest.reshape(-1, 28, 28), k=2, axes=(1, 2)).reshape(-1, 784)

# rotate 270 degrees counter-clockwise 
Xtest_r270 = np.rot90(Xtest.reshape(-1, 28, 28), k=3, axes=(1, 2)).reshape(-1, 784)

# visualize the 90 degrees rotation
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(Xtest_r90[i].reshape(28, 28), cmap=plt.cm.binary)
    plt.xlabel(class_names[Ytest[i]])
plt.show()

Provide the test accuracy of the base model, the CNN model  and the deeper CNN model on the three rotation test sets. Are the CNN and the deeper CNN still doing well?

In [ ]:
print("Rotation 90 degrees:")

test_loss, test_acc = base_model.evaluate(Xtest_r90,  Ytest, verbose=2)
print('Base model accuracy:', test_acc)
test_loss, test_acc = cnn_model.evaluate(Xtest_r90,  Ytest, verbose=2)
print('CNN model accuracy:', test_acc)
test_loss, test_acc = cnn_model_deeper.evaluate(Xtest_r90,  Ytest, verbose=2)
print('Deeper CNN model accuracy:', test_acc)

print("Rotation 180 degrees:")

test_loss, test_acc = base_model.evaluate(Xtest_r180,  Ytest, verbose=2)
print('Base model accuracy:', test_acc)
test_loss, test_acc = cnn_model.evaluate(Xtest_r180,  Ytest, verbose=2)
print('CNN model accuracy:', test_acc)
test_loss, test_acc = cnn_model_deeper.evaluate(Xtest_r270,  Ytest, verbose=2)
print('Deeper CNN model accuracy:', test_acc)

print("Rotation 270 degrees:")

test_loss, test_acc = base_model.evaluate(Xtest_r270,  Ytest, verbose=2)
print('Base model accuracy:', test_acc)
test_loss, test_acc = cnn_model.evaluate(Xtest_r270,  Ytest, verbose=2)
print('CNN model accuracy:', test_acc)
test_loss, test_acc = cnn_model_deeper.evaluate(Xtest_r270,  Ytest, verbose=2)
print('Deeper CNN model accuracy:', test_acc)